In [2]:
DF_PATH       = "../data/processed/2_cleaned_data.pkl"

ROLE_COLS      = ['DevType']
TECH_COLS      = ['LanguageHaveWorkedWith',
                  'DatabaseHaveWorkedWith',
                  #'PlatformHaveWorkedWith',
                  'WebframeHaveWorkedWith',
                  'MiscTechHaveWorkedWith',
                  'ToolsTechHaveWorkedWith',
                  'NEWCollabToolsHaveWorkedWith',
                  ]

MLFLOW_TRACKING_URI = '../models/mlruns'
MLFLOW_EXPERIMENT_NAME = "skills_jobs_stackoverflow"

LOG_PATH = "../models/temp/"
LOG_DATA_PKL    =  "data.pkl"
LOG_MODEL_PKL   =  "model.pkl"
LOG_METRICS_PKL =  "metrics.pkl"

In [3]:
# Load packages
import pandas as pd 
import numpy as np
import logging
import pickle
import random
import plotly 
import os
from pathlib import Path

import mlflow
from mlflow.tracking import MlflowClient

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline, FeatureUnion
from sklearn.feature_selection import VarianceThreshold
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import auc, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

from sklearn.decomposition import PCA, KernelPCA

from sklearn import tree
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from matplotlib import pyplot as plt

_________

### Functions

In [4]:
def calculate_quality(ground_truth, predictions, metric_function, sort_values=False):
    quality_scores = {}
    for col in predictions.columns:
        role_pred  = predictions[col].copy()
        role_truth = ground_truth[col].copy()
        quality_scores[col] = round(metric_function(role_truth, role_pred) * 100, 2)
        
    quality_scores = pd.Series(quality_scores.values(), index=quality_scores.keys())
    if sort_values:
        quality_scores = quality_scores.sort_values()
    
    return quality_scores


-------------

## Prep data

In [5]:
# Read Data
df = pd.read_pickle(DF_PATH)

### Balance classes

In [6]:
# Check the total samples of roles
roles_df = df["DevType"].copy()
roles_df.sum(axis=0)

Academic researcher                               1708
Data or business analyst                          1658
Data scientist or machine learning specialist     2460
Database administrator                            1210
DevOps specialist                                 3056
Developer, QA or test                             1135
Developer, back-end                              17084
Developer, desktop or enterprise applications     4845
Developer, embedded applications or devices       2138
Developer, front-end                              8932
Developer, full-stack                            20655
Developer, game or graphics                        899
Developer, mobile                                 4751
Engineer, data                                    1941
Scientist                                         1046
System administrator                              2069
dtype: int64

In [7]:
# Resample roles
samples_per_class = 1200
resampled_roles = []

for role_col in roles_df.columns:
    sub_df = roles_df.loc[roles_df[role_col] == 1].copy()

    if len(sub_df) < samples_per_class:
        # Upsample
        sub_df = sub_df.sample(samples_per_class, replace=True, random_state=0)
    else:
        # Downsample
        sub_df = sub_df.sample(samples_per_class, random_state=0)

    resampled_roles.append(sub_df)

In [8]:
# Construct dfs
roles_df = pd.concat(resampled_roles)
df = df.loc[roles_df.index].copy()

In [9]:
roles_df.sum(axis=0)

Academic researcher                              2280
Data or business analyst                         1965
Data scientist or machine learning specialist    2576
Database administrator                           1765
DevOps specialist                                2170
Developer, QA or test                            1514
Developer, back-end                              5710
Developer, desktop or enterprise applications    2690
Developer, embedded applications or devices      1773
Developer, front-end                             2614
Developer, full-stack                            5602
Developer, game or graphics                      1441
Developer, mobile                                2155
Engineer, data                                   2046
Scientist                                        1910
System administrator                             2110
dtype: int64

## Split to train and test

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(df.drop('DevType', axis=1),
                                                    df['DevType'],
                                                    random_state=0)

/var/folders/5f/pff47ncd2ql85c2cxp98vpk00000gn/T/ipykernel_7433/1966557603.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  X_train, X_test, Y_train, Y_test = train_test_split(df.drop('DevType', axis=1),


_________

# Train models

### Initialize MLflow

In [11]:
# Initialize client and experiment
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
client = MlflowClient()
mlflow.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)
exp = client.get_experiment_by_name(MLFLOW_EXPERIMENT_NAME)

### 1. Vanilla Forest 

In [12]:
rf_clf = make_pipeline(RobustScaler(),
                       PCA(n_components=0.95),
                       RandomForestClassifier(n_jobs=8,
                                              verbose=1,
                                              random_state=0))

rf_clf.fit(X_train.values, Y_train.values)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    3.5s finished


Pipeline(steps=[('robustscaler', RobustScaler()),
                ('pca', PCA(n_components=0.95)),
                ('randomforestclassifier',
                 RandomForestClassifier(n_jobs=8, random_state=0, verbose=1))])

In [13]:
# Evaluate on train set
predictions =  pd.DataFrame(rf_clf.predict(X_train.values),
                            columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.4s finished


In [14]:
# Evaluate on test set
predictions =  pd.DataFrame(rf_clf.predict(X_test.values),
                            columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)
mean_test_scores = test_scores.mean()

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.3s finished


In [15]:
print(test_scores.mean())
test_scores.sort_values("precision_score")

accuracy_score     92.645625
precision_score    94.251250
recall_score       50.111250
f1_score           64.450625
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
"Developer, full-stack",82.92,77.90,57.73,66.31
"Developer, back-end",78.96,84.58,36.28,50.78
"Developer, front-end",89.60,87.21,32.56,47.42
Data scientist or machine learning specialist,94.02,90.38,62.35,73.79
"Developer, mobile",94.62,93.40,55.62,69.72
"Developer, desktop or enterprise applications",91.02,93.44,35.46,51.41
Academic researcher,94.94,95.63,58.99,72.97
DevOps specialist,92.79,95.63,36.89,53.24
Scientist,97.21,97.88,74.55,84.63
"Developer, game or graphics",97.98,98.21,74.93,85.01


### Log

In [16]:
## Log
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices": X_test.index.tolist(),
                "features_names": X_train.columns.droplevel(0).tolist(),
                "targets_names": Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [17]:
# Model
model = {"model_description": "Random Forest: with PCA - Basic",
         "model_details": str(rf_clf),
         "model_object": rf_clf}

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [18]:
# Performance details
classes_metrics = {"train_scores": train_scores,
                   "test_scores": test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [19]:
# Start a new run and track
with mlflow.start_run(experiment_id=exp.experiment_id,
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)

    # Track metrics
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score)

______________

### 2. Hyper parameter tuning

In [20]:
hpt_rf_clf = make_pipeline(RobustScaler(),
                           PCA(),
                           RandomForestClassifier(n_jobs=8,
                                                  verbose=1,
                                                  random_state=0))

In [21]:
# list(hpt_rf_clf.get_params().keys())
tuned_parameters = [{
    'pca__n_components': [0.7, 0.85, 0.95],
    'randomforestclassifier__n_estimators': [250, 500],
    'randomforestclassifier__max_depth':    [3, 10, None],
}]


In [22]:
hpt_rf_clf = GridSearchCV(hpt_rf_clf, tuned_parameters)
hpt_rf_clf.fit(X_train.values, Y_train.values)

[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    2.7s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.5s finished
[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  56 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    1.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 250 out of 250 | elapsed:    0.6s finished
[Parallel(n_jobs=8)]: Using backend LokyBack

GridSearchCV(estimator=Pipeline(steps=[('robustscaler', RobustScaler()),
                                       ('pca', PCA()),
                                       ('randomforestclassifier',
                                        RandomForestClassifier(n_jobs=8,
                                                               random_state=0,
                                                               verbose=1))]),
             param_grid=[{'pca__n_components': [0.7, 0.85, 0.95],
                          'randomforestclassifier__max_depth': [3, 10, None],
                          'randomforestclassifier__n_estimators': [250, 500]}])

In [23]:
hpt_rf_clf.best_params_

{'pca__n_components': 0.7,
 'randomforestclassifier__max_depth': None,
 'randomforestclassifier__n_estimators': 500}

In [24]:
# Evaluate on test set
predictions =  pd.DataFrame(hpt_rf_clf.predict(X_train.values),
                            columns=Y_train.columns)
train_scores = {score.__name__: calculate_quality(Y_train, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
train_scores = pd.concat(train_scores,axis=1)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.9s finished


In [25]:
# Evaluate on test set
predictions =  pd.DataFrame(hpt_rf_clf.predict(X_test.values),
                            columns=Y_test.columns)
test_scores = {score.__name__: calculate_quality(Y_test, predictions, score) 
                for score in [accuracy_score, precision_score, recall_score, f1_score]}
test_scores = pd.concat(test_scores,axis=1)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    1.4s finished


In [26]:
print(test_scores.mean())
test_scores

accuracy_score     92.748750
precision_score    90.772500
recall_score       52.986875
f1_score           66.008125
dtype: float64


,accuracy_score,precision_score,recall_score,f1_score
Academic researcher,94.79,91.35,60.79,73.00
Data or business analyst,94.52,92.80,50.10,65.07
Data scientist or machine learning specialist,94.08,85.55,67.59,75.52
Database administrator,94.96,98.54,45.93,62.65
DevOps specialist,93.10,90.44,42.51,57.83
"Developer, QA or test",96.52,99.14,58.23,73.37
"Developer, back-end",79.29,77.76,43.11,55.47
"Developer, desktop or enterprise applications",90.90,89.02,36.55,51.82
"Developer, embedded applications or devices",94.88,89.72,46.15,60.95
"Developer, front-end",89.58,82.37,35.17,49.29


## Log

In [27]:
# Data details
data_details = {"data_path": DF_PATH,
                "training_indices": X_train.index.tolist(),
                "test_indices":     X_test.index.tolist(), 
                "features_names":   X_train.columns.droplevel(0).tolist(),
                "targets_names":    Y_train.columns.tolist()}

with open(os.path.join(LOG_PATH, LOG_DATA_PKL), "wb") as output_file:
    pickle.dump(data_details, output_file)

In [28]:
# Model
model = {"model_description": "Random Forest: with PCA + Hyperparamter tuning",
         "model_details": str(hpt_rf_clf),
         "model_object": hpt_rf_clf} 

with open(os.path.join(LOG_PATH, LOG_MODEL_PKL), "wb") as output_file:
    pickle.dump(model, output_file)

In [29]:
# Preformance details
classes_metrics = {"train_scores": train_scores, 
                   "test_scores":  test_scores}

with open(os.path.join(LOG_PATH, LOG_METRICS_PKL), "wb") as output_file:
    pickle.dump(classes_metrics, output_file)

In [30]:
# Start a new run and track 
with mlflow.start_run(experiment_id=exp.experiment_id, 
                      run_name=model["model_description"]):
    # Log pickles
    mlflow.log_artifacts(LOG_PATH)  
    
    # Track metrics 
    for metric, score in mean_test_scores.items():
        mlflow.log_metric(metric, score) 
    